# **Tarea 2. Encoder-Decoder Architectures**
# Traductor Inglés-Español
Guillermo Segura Gómez

## **Instrucciones**

1. Por cada ejercicio entregar un notebook con su nombre (e.g., adrian_lopez.ipynb)
2. Está prohibido compartirse código entre los estudiantes, aunque pueden comentar entre ellos las soluciones propuestas y dificultades que tengan.
3. Se pueden usar todas las funciones disponibles en PyTorch, excepto aquellas en las que explicitamente se indique en el problema.
4. Debe documentar su código con comentarios y breves discusiones de tal forma que se ve su proceso de razonamiento y facilite la evaluación para el profesor.
5. Debe proporcionar ejemplos del funcionamiento de su arquitectura. Ejemplos dónde se vea un buen resultado y otros dónde no. Deberá discutirlo brevemente y mostrar su razonamiento y criterio.
6. Muy importante: no debe usar códigos hechos por otros autores en internet. Esto será altamente penalizado.

## **Machine Translation**

1. (35pts) Haga la versión PyTorch inspirada en la arquitectura del notebook adjunto C4_W1_Assignment del curso de Natural Language Processing de Coursera.
2. (15pts) Haga un nuevo notebook dónde construya un traductor Inglés a Español.

Vamos a construir un traductor de inglés - aleman utilizando la librería de PyTorch basándonos en el curso de NLP de Coursera. Es un modelo neuronal basado en una red tipo LSTM. Es importante no solo encontrar la traducción de palabra a palabra, sino poder desambiguar el lenguaje para darle un mejor sentido a la traducción.

## Importanción y preprocesado de datos

Como se menciono antes, vamos a utilizar pyTorch para lograr el modelo. :D

Para el modelo de traducción inglés-español usamos el dataset [opus100](https://opus.nlpl.eu/opus-100.php). Además reciclamos el código de la parte 1.

In [1]:
import numpy as np
import os
import re
import nltk
import random
from nltk.tokenize import word_tokenize
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from concurrent.futures import ThreadPoolExecutor
from nltk.corpus import stopwords
from collections import Counter

from sklearn.model_selection import train_test_split
from sklearn.model_selection import train_test_split

import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
from transformers import MarianTokenizer

In [2]:
!pip install datasets

INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.1 MB/s eta 0:00:00


In [3]:
from datasets import load_dataset

# Cargamos el dataset opus100 inglés-español
dataset = load_dataset("opus100", "en-es", split='train')

# Ver un ejemplo del dataset
print(dataset[0])

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/65.4k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/237k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/99.6M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/238k [00:00<?, ?B/s]

Generating test split:   0%|          | 0/2000 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/1000000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2000 [00:00<?, ? examples/s]

{'translation': {'en': "It was the asbestos in here, that's what did it!", 'es': 'Fueron los asbestos aquí. ¡Eso es lo que ocurrió!'}}


In [4]:
# Mostramos el tamaño del dataset
print(f"Tamaño del dataset: {len(dataset)}")

# Estructura del dataset
print(f"Estructura del dataset: {dataset.features}")

Tamaño del dataset: 1000000
Estructura del dataset: {'translation': Translation(languages=['en', 'es'], id=None)}


Ahora necesitamos procesar el dataset. De hugginface se carga como tipo json. Vamos a convertir a una lista tokenizada con la cual podamos alimentar los modelo que vayamos a implementar.

Son muchos ejemplos, los limitamos para poder procesar mejor el modelo.

In [5]:
def preprocess_function(examples, num_examples=None):
    if num_examples:
        examples = examples.select(range(num_examples))

    # Extraer las oraciones en inglés y alemán
    inputs = [ex['en'] for ex in examples['translation']]
    targets = [ex['es'] for ex in examples['translation']]

    return inputs, targets

inputs, targets = preprocess_function(dataset, num_examples=10000)

# Resultados
print(inputs[:5])
print(targets[:5])

["It was the asbestos in here, that's what did it!", "I'm out of here.", 'One time, I swear I pooped out a stick of chalk.', 'And I will move, do you understand me?', '- Thank you, my lord.']
['Fueron los asbestos aquí. ¡Eso es lo que ocurrió!', 'Me voy de aquí.', 'Una vez, juro que cagué una barra de tiza.', 'Y prefiero mudarme, ¿Entiendes?', '- Gracias.']


In [6]:
len(inputs)

10000

Ahora que tenemos el dataset cargado en lista lo tokenizamos. Utiizamos las stopwords tanto del inglés como el alemán.

In [7]:
# Descargar las stopwords
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [8]:
# Cargamos las stopwords
stop_words_en = set(nltk.corpus.stopwords.words('english'))
stop_words_sp = set(nltk.corpus.stopwords.words('spanish'))

# Combinar stopwords en un solo conjunto
stop_words = stop_words_en.union(stop_words_sp)

# Función para preprocesar un solo tweet
def procesar_tweet(tweet):
    # Convertir a minúsculas
    tweet = tweet.lower()

    # Tokenizar
    tokens = word_tokenize(tweet)

    # Filtrar stopwords
    tokens = [word for word in tokens if word not in stop_words]

    # Quitar carácteres especiales
    tokens = [re.sub(r'[^\w\s]', '', word) for word in tokens]

    # Eliminar palabras vacías
    tokens = [word for word in tokens if word]

    return tokens

# Función para preprocesar múltiples tweets de manera paralela
def preprocesar(corpus, num_workers=4):
    # Usar ThreadPoolExecutor para procesar los tweets en paralelo
    with ThreadPoolExecutor(max_workers=num_workers) as executor:
        # Ejecutar el procesamiento en paralelo y recolectar los resultados
        futures = [executor.submit(procesar_tweet, tweet) for tweet in corpus]
        preprocesado = [future.result() for future in futures]  # Esperar a que los resultados estén listos

    return preprocesado

In [9]:
# Separamos el dataset
inputs_train, inputs_test, targets_train, targets_test = train_test_split(
    inputs, targets, test_size=0.2, random_state=42)

print(f"Tamaño del conjunto de entrenamiento: {len(inputs_train)}")
print(f"Tamaño del conjunto de prueba: {len(inputs_test)}")

Tamaño del conjunto de entrenamiento: 8000
Tamaño del conjunto de prueba: 2000


In [10]:
inputs_train_tokenized = preprocesar(inputs_train)
inputs_test_tokenized = preprocesar(inputs_test)
targets_train_tokenized = preprocesar(targets_train)
targets_test_tokenized = preprocesar(targets_test)

In [11]:
inputs_train_tokenized[0]

['also',
 'contradiction',
 'united',
 'nations',
 'regulations',
 'rules',
 'financial',
 'rule',
 '10518',
 'contract',
 'formally',
 'awarded',
 'signed',
 'national',
 'archives',
 'netherlands',
 'restoration',
 'work']

## Modelo

Una vez con los datos preprocesados, definimos un modelo tipo encoder - decoder como se mostro en el ejemplo. Además necesitamos convertir a embeddings las palabras. Para esto es necesario construir un vocabulario para convertir indices a palabras.

In [12]:
# Hiperparámetros
_BATCH_SIZE_ = 10
_NUM_WORKERS_ = 0
_EMBEDDING_DIM_ = 128
_HIDDEN_DIM_WORD_ = 32
_HIDDEN_DIM_TWEET_  = 32
_NUM_EPOCHS_    = 10
_LEARNING_RATE_ = 0.001
_OUTPUT_DIM_ = 7
_NUM_LAYERS_ = 2
_DROPOUT_ = 0.5

In [13]:
def build_vocab(corpus):
    # Contar las palabras en el corpus (cada 'sentence' es una lista de palabras ya tokenizada)
    word_counts = Counter([word for sentence in corpus for word in sentence])

    # Crear el vocabulario a partir de la frecuencia de las palabras, comenzando en 4
    # para dejar espacio para los tokens especiales
    vocab = {word: idx for idx, (word, _) in enumerate(word_counts.items(), start=4)}  # Comienza en 4 para dejar espacio a los tokens especiales

    # Agregar tokens especiales al vocabulario
    vocab['<PAD>'] = 0  # El token de relleno será el índice 0
    vocab['<sos>'] = 1  # Token de inicio
    vocab['<eos>'] = 2  # Token de fin
    vocab['<unk>'] = 3  # Token de palabras desconocidas

    return vocab

# Construir el vocabulario en inglés
vocab_en = build_vocab(inputs_train_tokenized)
print(f"Tamaño del vocabulario inglés: {len(vocab_en)}")
print(vocab_en)

# Construir el vocabulario en alemán
vocab_es = build_vocab(targets_train_tokenized)
print(f"Tamaño del vocabulario alemán: {len(vocab_es)}")
print(vocab_es)

Tamaño del vocabulario inglés: 11629
{'also': 4, 'contradiction': 5, 'united': 6, 'nations': 7, 'regulations': 8, 'rules': 9, 'financial': 10, 'rule': 11, '10518': 12, 'contract': 13, 'formally': 14, 'awarded': 15, 'signed': 16, 'national': 17, 'archives': 18, 'netherlands': 19, 'restoration': 20, 'work': 21, 'us': 22, 'left': 23, 'know': 24, 'ay': 25, 'wait': 26, 'yes': 27, 'wondering': 28, 'way': 29, 'could': 30, 'look': 31, 'calm': 32, 'people': 33, 'government': 34, 'first': 35, 'many': 36, 'ships': 37, 'kidding': 38, 'get': 39, 'home': 40, 're': 41, 'racing': 42, '1000000': 43, 'house': 44, '21': 45, 'december': 46, '2006': 47, 'present': 48, 'clinical': 49, 'case': 50, 'less': 51, 'frequent': 52, 'malignancy': 53, 'even': 54, 'four': 55, 'centimeters': 56, 'dilated': 57, 'class': 58, 'forum': 59, 'hosted': 60, 'friends': 61, 'goodwill': 62, 'cameroon': 63, 'action': 64, 'life': 65, 'afl': 66, '5': 67, '1': 68, 'team': 69, 'currently': 70, 'outreach': 71, 'kenya': 72, 'mean': 73, 

In [14]:
# Convertir los tweets tokenizados en índices numéricos
def tokens_to_indices(corpus, vocab):
    return [[vocab.get(word, 0) for word in tweet] for tweet in corpus]

# Convertir los datos de entrenamiento y prueba en índices numéricos
inputs_train_indices = tokens_to_indices(inputs_train_tokenized, vocab_en)
inputs_test_indices = tokens_to_indices(inputs_test_tokenized, vocab_en)
targets_train_indices = tokens_to_indices(targets_train_tokenized, vocab_es)
targets_test_indices = tokens_to_indices(targets_test_tokenized, vocab_es)

In [15]:
print(inputs_train_indices[3])
print(inputs_train_tokenized[3])

print(targets_train_indices[3])
print(targets_train_tokenized[3])

[27, 28, 29, 30, 31, 29]
['yes', 'wondering', 'way', 'could', 'look', 'way']
[17, 18, 19, 20, 21]
['preguntaba', 'alguna', 'forma', 'mires', 'lado']


Ahora creamos los dataset y dataloaders.

Es necesario constuir además un vocabulario inverso para que la tarea de traducción sea completada

In [16]:
vocab_inv_en = {index: word for word, index in vocab_en.items()}
vocab_inv_es = {index: word for word, index in vocab_es.items()}
print(vocab_inv_en)
print(vocab_inv_es)

{4: 'also', 5: 'contradiction', 6: 'united', 7: 'nations', 8: 'regulations', 9: 'rules', 10: 'financial', 11: 'rule', 12: '10518', 13: 'contract', 14: 'formally', 15: 'awarded', 16: 'signed', 17: 'national', 18: 'archives', 19: 'netherlands', 20: 'restoration', 21: 'work', 22: 'us', 23: 'left', 24: 'know', 25: 'ay', 26: 'wait', 27: 'yes', 28: 'wondering', 29: 'way', 30: 'could', 31: 'look', 32: 'calm', 33: 'people', 34: 'government', 35: 'first', 36: 'many', 37: 'ships', 38: 'kidding', 39: 'get', 40: 'home', 41: 're', 42: 'racing', 43: '1000000', 44: 'house', 45: '21', 46: 'december', 47: '2006', 48: 'present', 49: 'clinical', 50: 'case', 51: 'less', 52: 'frequent', 53: 'malignancy', 54: 'even', 55: 'four', 56: 'centimeters', 57: 'dilated', 58: 'class', 59: 'forum', 60: 'hosted', 61: 'friends', 62: 'goodwill', 63: 'cameroon', 64: 'action', 65: 'life', 66: 'afl', 67: '5', 68: '1', 69: 'team', 70: 'currently', 71: 'outreach', 72: 'kenya', 73: 'mean', 74: 'anything', 75: 'dear', 76: 'diar

In [17]:
def indices_to_sequence(indices, vocab_inv):
    """
    Convierte una secuencia de índices a una secuencia de palabras usando el vocabulario inverso.
    """
    return [vocab_inv.get(idx, '<unk>') for idx in indices]  # Reemplaza con <unk> si el índice no está en vocab_inv

In [18]:
# Probamos la función

indices_example = inputs_train_indices[3]
sequence_example = indices_to_sequence(indices_example, vocab_inv_en)

target_example = targets_train_indices[3]
target_sequence_example = indices_to_sequence(target_example, vocab_inv_es)

print(sequence_example)
print(target_sequence_example)

['yes', 'wondering', 'way', 'could', 'look', 'way']
['preguntaba', 'alguna', 'forma', 'mires', 'lado']


In [19]:
class TranslationDataset(Dataset):
    def __init__(self, inputs, targets):
        self.inputs = inputs
        self.targets = targets

    def __len__(self):
        return len(self.inputs)

    def __getitem__(self, idx):
        return self.inputs[idx], self.targets[idx]

In [20]:
# Creamos los dataset
train_dataset = TranslationDataset(inputs_train_indices, targets_train_indices)
test_dataset = TranslationDataset(inputs_test_indices, targets_test_indices)

Definimos una función **collate_fn** para paddear las secuencias.

In [21]:
def collate_fn(batch, pad_idx=0):

    # Separar inputs (secuencias de origen) y targets (secuencias objetivo)
    inputs, targets = zip(*batch)

    # Convertir las listas de secuencias en tensores
    inputs_tensor = []
    for seq in inputs:
        inputs_tensor.append(torch.tensor(seq, dtype=int))
    targets_tensor = []
    for seq in targets:
        targets_tensor.append(torch.tensor(seq, dtype=int))

    # Aplicar padding dinámico en las secuencias de entrada y salida
    inputs_padded = nn.utils.rnn.pad_sequence(inputs_tensor, batch_first=True, padding_value=pad_idx)
    targets_padded = nn.utils.rnn.pad_sequence(targets_tensor, batch_first=True, padding_value=pad_idx)

    return inputs_padded, targets_padded

In [22]:
# Crear el DataLoader
train_loader = DataLoader(train_dataset,
                          batch_size=_BATCH_SIZE_,
                          shuffle=True,
                          collate_fn=collate_fn)

test_loader = DataLoader(test_dataset,
                         batch_size=_BATCH_SIZE_,
                         shuffle=False,
                         collate_fn=collate_fn)

Definimos ahora un modelo tipo encoder-decoder.

In [23]:
class Encoder(nn.Module):
    def __init__(self, input_dim, emb_dim, hid_dim, dropout):
        super(Encoder, self).__init__()

        # Capa de embeddings
        self.embedding = nn.Embedding(input_dim, emb_dim)

        # LSTM con una sola capa
        self.rnn = nn.LSTM(emb_dim, hid_dim, batch_first=True)

        # Capa de dropout
        self.dropout = nn.Dropout(dropout)

    def forward(self, src):
        # Embeddings de la secuencia de entrada
        embedded = self.dropout(self.embedding(src))  # (batch_size, src_len, emb_dim)

        # Pasar los embeddings por la LSTM
        outputs, hidden = self.rnn(embedded)  # outputs -> (batch_size, src_len, hid_dim)

        return hidden  # hidden -> ((h_n, c_n) -> tupla con el estado oculto y de memoria)

class Decoder(nn.Module):
    def __init__(self, output_dim, emb_dim, hid_dim, dropout):
        super(Decoder, self).__init__()

        # Capa de embeddings
        self.embedding = nn.Embedding(output_dim, emb_dim)

        # LSTM con una sola capa
        self.rnn = nn.LSTM(emb_dim, hid_dim, batch_first=True)

        # Capa lineal para transformar el estado oculto en predicciones de palabras
        self.fc_out = nn.Linear(hid_dim, output_dim)

        # Capa de dropout
        self.dropout = nn.Dropout(dropout)

    def forward(self, trg, hidden):
        # Embedding del token de entrada
        embedded = self.dropout(self.embedding(trg))  # (batch_size, 1, emb_dim)

        # Pasar el embedding por la LSTM
        output, hidden = self.rnn(embedded, hidden)  # output -> (batch_size, 1, hid_dim)

        # Predicción con la capa final
        prediction = self.fc_out(output.squeeze(1))  # prediction -> (batch_size, output_dim)

        return prediction, hidden


class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super(Seq2Seq, self).__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.device = device

    def forward(self, src, trg):
        # Paso por el encoder
        hidden = self.encoder(src)

        # La primera entrada al decoder es el token <sos>
        input = trg[:, 0].unsqueeze(1)  # (batch_size, 1)

        # Guardar las predicciones
        outputs = torch.zeros(trg.size(0), trg.size(1), self.decoder.fc_out.out_features).to(self.device)

        # Decodificación paso a paso (sin teacher forcing)
        for t in range(1, trg.shape[1]):
            output, hidden = self.decoder(input, hidden)  # Decodificación

            # Guardar la predicción
            outputs[:, t, :] = output

            # El siguiente token es siempre la predicción más probable
            input = output.argmax(1).unsqueeze(1)  # Predicción más probable (batch_size, 1)

        return outputs

In [24]:
# Definimos el dispositivo
if torch.cuda.is_available():
    device = torch.device("cuda")
elif torch.backends.mps.is_available():
    device = torch.device("mps")
else:
    device = torch.device("cpu")
print("El dispositivo es:", device)

El dispositivo es: cuda


In [25]:
# Definimos el tamaño del vocabulario en inglés y alemán
vocab_en_size = len(vocab_en)
vocab_de_size = len(vocab_es)

encoder = Encoder(input_dim=vocab_en_size,
                  emb_dim=_EMBEDDING_DIM_,
                  hid_dim=_HIDDEN_DIM_WORD_,
                  dropout=_DROPOUT_)

decoder = Decoder(output_dim=vocab_de_size,
                  emb_dim=_EMBEDDING_DIM_,
                  hid_dim=_HIDDEN_DIM_WORD_,
                  dropout=_DROPOUT_)

# Instanciamos el modelo Seq2Seq
model = Seq2Seq(encoder, decoder, device).to(device)

In [26]:
print(model)

Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(11629, 128)
    (rnn): LSTM(128, 32, batch_first=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (decoder): Decoder(
    (embedding): Embedding(14499, 128)
    (rnn): LSTM(128, 32, batch_first=True)
    (fc_out): Linear(in_features=32, out_features=14499, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
)


Definimos una función para comparar la similaridad.

In [27]:
def bleu_score(candidate_sequence, reference_sequence):
    """
    Calcula el BLEU score entre dos secuencias de palabras.
    """
    reference_sequence = [reference_sequence]
    smoothing = SmoothingFunction().method4
    return sentence_bleu(reference_sequence, candidate_sequence, smoothing_function=smoothing)

In [28]:
# Definir el optimizador y la función de pérdida
optimizer = torch.optim.Adam(model.parameters(), lr=_LEARNING_RATE_)
criterion = nn.CrossEntropyLoss()

log_interval = 50

In [30]:
for epoch in range(_NUM_EPOCHS_):
    model.train()
    total_loss = 0
    total_bleu = 0
    total_samples = 0  # Contar el número de secuencias procesadas

    for batch_idx, (src, trg) in enumerate(train_loader):
        optimizer.zero_grad()

        # Pasar las secuencias al dispositivo
        src, trg = src.to(device), trg.to(device)

        # Forward pass: el decoder usa src y predice trg
        outputs = model(src, trg)

        # Ajustar las dimensiones para la función de pérdida
        outputs = outputs[:, 1:].reshape(-1, outputs.shape[-1])  # Remover el token <sos> de las predicciones y aplanar
        src_flat = trg[:, 1:].contiguous().view(-1)  # Remover el token <sos> y aplanar

        # Calcular la pérdida
        loss = criterion(outputs, src_flat)
        loss.backward()
        optimizer.step()

        # Acumular la pérdida total
        total_loss += loss.item()

        # Obtener las predicciones con argmax
        predicted = outputs.argmax(1).view(trg.size(0), -1)  # Predicciones por lote (restauramos batch_size)

        # Calcular Jaccard en lote
        for i in range(src.size(0)):  # Iterar sobre el lote
            # Como removimos el <sos> en trg_flat, lo removemos también de la secuencia predicha
            candidate = predicted[i].tolist()  # Predicción generada por el modelo
            reference = src[i, 1:].tolist()  # Secuencia de referencia sin el token <sos>

            # Convertimos a palabras
            candidate_sequence = indices_to_sequence(candidate, vocab_inv_en)  # Inglés (predicción)
            reference_sequence = indices_to_sequence(reference, vocab_inv_en)  # Inglés (referencia)

            bleu = bleu_score(candidate_sequence, reference_sequence)
            total_bleu += bleu
            total_samples += 1

            # Imprimir un ejemplo de la traducción
            # if i == 0 and batch_idx % log_interval == 0:  # Imprimir solo para el primer ejemplo del lote
            #     print(f"Ejemplo de traducción:")
            #     print(f"Entrada (Inglés): {' '.join(reference_sequence)}")
            #     print(f"Predicción (Español): {' '.join(candidate_sequence)}")
            #     print(f"Bleu Similarity: {bleu:.4f}")

        # Imprimir cada `log_interval` iteraciones
        if batch_idx % log_interval == 0:
            print(f"Epoch {epoch+1}, Batch {batch_idx+1}/{len(train_loader)}, Loss: {loss.item():.4f}")
            print(f"Bleu Similarity: {bleu:.4f}")


    # Calcular la pérdida promedio y la similaridad de Jaccard promedio
    avg_loss = total_loss / len(train_loader)
    avg_bleu = total_bleu / total_samples

    print(f"Epoch {epoch+1} completed, Avg Loss: {avg_loss:.4f}, Avg Bleu: {avg_bleu:.4f}")

Epoch 1, Batch 1/800, Loss: 2.3069
Bleu Similarity: 0.8386
Epoch 1, Batch 51/800, Loss: 1.3465
Bleu Similarity: 0.0000
Epoch 1, Batch 101/800, Loss: 2.0992
Bleu Similarity: 0.8465
Epoch 1, Batch 151/800, Loss: 2.9446
Bleu Similarity: 0.9306
Epoch 1, Batch 201/800, Loss: 3.2874
Bleu Similarity: 0.5956
Epoch 1, Batch 251/800, Loss: 2.1876
Bleu Similarity: 0.9391
Epoch 1, Batch 301/800, Loss: 1.6051
Bleu Similarity: 0.6610
Epoch 1, Batch 351/800, Loss: 2.0288
Bleu Similarity: 0.7861
Epoch 1, Batch 401/800, Loss: 1.7656
Bleu Similarity: 0.9466
Epoch 1, Batch 451/800, Loss: 2.5755
Bleu Similarity: 0.6918
Epoch 1, Batch 501/800, Loss: 3.4810
Bleu Similarity: 0.8507
Epoch 1, Batch 551/800, Loss: 2.1577
Bleu Similarity: 0.0000
Epoch 1, Batch 601/800, Loss: 2.1552
Bleu Similarity: 0.0000
Epoch 1, Batch 651/800, Loss: 1.9174
Bleu Similarity: 0.4172
Epoch 1, Batch 701/800, Loss: 2.1962
Bleu Similarity: 0.1482
Epoch 1, Batch 751/800, Loss: 2.8231
Bleu Similarity: 0.1161
Epoch 1 completed, Avg Loss

Para probar el modelo es necesario crear un vocabulario inverso.

In [31]:
def test_model(model, test_loader, vocab_inv, criterion=None, device='cpu'):
    """
    Prueba el modelo en un conjunto de prueba y devuelve las métricas y traducciones.

    """
    model.eval()  # Poner el modelo en modo evaluación
    total_loss = 0
    total_samples = 0
    predictions = []

    with torch.no_grad():
        for batch_idx, (src, trg) in enumerate(test_loader):
            # Pasar los datos al dispositivo (CPU o GPU)
            src, trg = src.to(device), trg.to(device)

            # Forward pass
            outputs = model(src, trg)

            # Si criterion está disponible, calcular la pérdida
            if criterion:
                outputs_flat = outputs.view(-1, outputs.shape[-1])
                trg_flat = trg.view(-1)
                loss = criterion(outputs_flat, trg_flat)
                total_loss += loss.item() * trg.size(0)
                total_samples += trg.size(0)

            # Obtener las predicciones de los índices con argmax
            predicted_indices = outputs.argmax(dim=-1)

            # Convertir los índices predichos a palabras usando vocab_inv
            for idx_seq in predicted_indices:
                predicted_words = [vocab_inv.get(idx.item(), '<unk>') for idx in idx_seq]
                predictions.append(" ".join(predicted_words))

            # Imprimir las primeras predicciones de muestra
            if batch_idx == 0:
                print(f"Ejemplo de predicción: {predictions[0]}")

    avg_loss = total_loss / total_samples if criterion else None

    # Devolver la pérdida promedio (si se calculó) y las predicciones generadas
    return avg_loss, predictions

In [32]:
# Ejemplo de uso de test_model
avg_loss, predicciones = test_model(model, test_loader, vocab_inv_en, criterion=nn.CrossEntropyLoss(), device=device)

if avg_loss is not None:
    print(f"Pérdida promedio en el conjunto de prueba: {avg_loss:.4f}")

# Mostrar algunas predicciones
for i, pred in enumerate(predicciones[:5]):  # Mostrar las primeras 5 predicciones
    print(f"Predicción {i+1}: {pred}")

Ejemplo de predicción: <PAD> matters forgive forgive forgive forgive <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>
Pérdida promedio en el conjunto de prueba: 2.5855
Predicción 1: <PAD> matters forgive forgive forgive forgive <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>
Predicción 2: <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>
Predicción 3: <PAD> <PAD> <PAD> <

No le fue tan bien a este otro :(

De igual manera, se atribuye a la fata de mas datos o un mejor modelo.

Aunque por ejemplo la predicción 1 funcionó bien, entonces puede que funcione para algunas predicciones.